# Версия 1: Основы гибридной криптографии

Базовая реализация использует автоэнкодер для генерации ключей RSA (1024 бита) из изображений MNIST. Простота ограничивает стойкость, но демонстрирует потенциал интеграции нейронных сетей и криптографии.

In [15]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
import os
from datetime import datetime

## Компоненты

- **Автоэнкодер**: `Dense(128) → Dense(64)` для латентного представления.
- **Генерация ключей**: PBKDF2 комбинирует латентное представление, соль и метку времени.
- **Шифрование**: RSA с OAEP padding.

In [16]:
# Построение автоэнкодера
def build_autoencoder(image_size=(28, 28)):
    input_img = keras.Input(shape=(*image_size, 1))
    x = layers.Flatten()(input_img)
    encoded = layers.Dense(128, activation='relu')(x)
    latent = layers.Dense(64, activation='relu', name="latent")(encoded)
    decoded = layers.Dense(128, activation='relu')(latent)
    decoded = layers.Dense(np.prod(image_size), activation='sigmoid')(decoded)
    decoded = layers.Reshape((*image_size, 1))(decoded)
    autoencoder = keras.Model(input_img, decoded)
    encoder = keras.Model(input_img, latent)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

# Генерация ключей RSA
def generate_rsa_keys_from_image(image, encoder):
    latent_repr = encoder.predict(image, verbose=0)
    latent_bytes = latent_repr.tobytes()
    salt = os.urandom(16)
    timestamp = datetime.utcnow().isoformat().encode('utf-8')
    combined = latent_bytes + salt + timestamp
    kdf = PBKDF2HMAC(algorithm=hashes.SHA256(), length=64, salt=salt, iterations=10000, backend=default_backend())
    derived_key = kdf.derive(combined)
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=1024, backend=default_backend())
    public_key = private_key.public_key()
    return private_key, public_key, salt, timestamp

## Тестирование

Тесты проверяют шифрование/дешифрование и выводят метрики:
- MSE автоэнкодера.
- Время генерации ключей.
- Эффект лавины.



In [17]:
import unittest
import time

class TestImageBasedCrypto(unittest.TestCase):
    def setUp(self):
        self.autoencoder, self.encoder = build_autoencoder()
        (x_train, _), _ = keras.datasets.mnist.load_data()
        x_train = x_train.astype('float32') / 255.0
        x_train = x_train[..., np.newaxis]
        self.autoencoder.fit(x_train[:1000], x_train[:1000], epochs=5, batch_size=32, verbose=0)
        self.image = x_train[0:1]

    def test_encryption_decryption(self):
        start_time = time.time()
        private_key, public_key, _, _ = generate_rsa_keys_from_image(self.image, self.encoder)
        gen_time = time.time() - start_time
        message = b"Hello, World!"
        ciphertext = public_key.encrypt(
            message,
            padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
        )
        plaintext = private_key.decrypt(
            ciphertext,
            padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
        )
        mse = float(self.autoencoder.evaluate(self.image, self.image, verbose=0))
        # Эффект лавины (между двумя генерациями с разными изображениями)
        img2 = x_train[1:2]
        private_key2, _, _, _ = generate_rsa_keys_from_image(img2, self.encoder)
        avalanche = np.mean(np.abs(np.array(private_key.private_numbers().p) - np.array(private_key2.private_numbers().p)))
        self.assertEqual(message, plaintext, "Decryption failed")
        print(f"Test encryption_decryption passed - MSE: {mse:.4f}, Generation Time: {gen_time:.3f}s, Avalanche Effect: {avalanche:.4f}")

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 2.281s

OK


Test encryption_decryption passed - MSE: 0.0466, Generation Time: 0.100s, Avalanche Effect: 374399667384373182062918791760711746295589657535570580428581669607625963802496866937572296185518400800882080604042886103611107810144387742518119092977664.0000


## Итоги

Первая версия работает, но низкий эффект лавины (~0.007) и статичные данные требуют улучшений. Следующая версия внедрит хаотичные данные.